# ESN-PdM Framework ML models
This notebook contains the implementation of the ESN-PdM framework's machine learning models. These models are developed using the `tensorflow` library and trained with a dataset specifically generated for this project. The data is located in the `dataset` folder of this repository and consists of several files, each corresponding to one of the four classes the models are trained to predict. The data was generated using a BMI270 IMU and an ESP32, hence the features include the raw accelerometer and gyroscope readings from the IMU.

### Imports, Constants and Converters
First, we import the necessary libraries and define some constants and converters that will be used throughout the notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import json
import os
import tempfile
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from sklearn.model_selection import train_test_split

INPUT_DATA_PATH = 'datos/'
BUILD_FOLDER = "build"

LABELS = {
    "Good": 0,
    "Acceptable": 1,
    "Unacceptable": 2,
    "Bad": 3
}

# --- Accelerometer Constants ---
G_MS2 = 9.80665
MAX_INT_VALUE_SENSOR = 32768.0
ACC_RAW_TO_MS2 = (G_MS2 / MAX_INT_VALUE_SENSOR)
SENSOR_ACC_RANGE = 2 # 8g

# --- Gyroscope Constants ---
SENSOR_GYR_RANGE = 250.0
PI = 3.14159265359
GYR_RAW_TO_RADS = (PI / 180.0) / MAX_INT_VALUE_SENSOR

# --- Converters ---
convert_raw_acc_to_ms2 = lambda raw: (pow(2, SENSOR_ACC_RANGE + 1) * ACC_RAW_TO_MS2) * raw
convert_raw_gyr_to_rads = lambda raw: SENSOR_GYR_RANGE * GYR_RAW_TO_RADS * raw

2024-09-25 17:28:35.449401: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-25 17:28:35.470313: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 17:28:35.470338: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 17:28:35.470350: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 17:28:35.474780: I tensorflow/core/platform/cpu_feature_g

### Data Loading and Preprocessing
Next, we load the data and preprocess it. The data is loaded from the files in the `dataset` folder and preprocessed to be used in the models. The preprocessing steps include cleaning null values, converting the raw data to the appropriate format and normalizing the data.

In [2]:
SEQ_LENGTH = 500

with open(INPUT_DATA_PATH + 'column_names.json') as f:
    column_names = json.load(f)

labeled_data_frames = {l: [] for l in LABELS.values()}
for filename in os.listdir(INPUT_DATA_PATH):
    for label in LABELS.values():
        if filename.startswith(str(label)) and filename.endswith('.csv'):
            df = pd.read_csv(INPUT_DATA_PATH + filename, names=column_names, sep=';')
            
            # Convert the timestamp column to a datetime object
            df['timestamp'] = pd.to_datetime(df['timestamp'])
            
            # Create a mask for rows where acc_x, acc_y, and acc_z are all 0
            mask_acc_zero = (df['acc_x'] == 0) & (df['acc_y'] == 0) & (df['acc_z'] == 0)

            # Create a mask for rows where gyro_x, gyro_y, and gyro_z are all 0
            mask_gyro_zero = (df['gyro_x'] == 0) & (df['gyro_y'] == 0) & (df['gyro_z'] == 0)

            # Combine the masks to identify rows where either condition is true
            mask_either_zero = mask_acc_zero | mask_gyro_zero

            # Filter out the rows from the DataFrame
            df = df[~mask_either_zero]

            # Convert the raw accelerometer data to m/s^2
            df['acc_x'] = df['acc_x'].apply(convert_raw_acc_to_ms2).astype("float32")
            df['acc_y'] = df['acc_y'].apply(convert_raw_acc_to_ms2).astype("float32")
            df['acc_z'] = df['acc_z'].apply(convert_raw_acc_to_ms2).astype("float32")

            # Convert the raw gyroscope data to rad/s
            df['gyro_x'] = df['gyro_x'].apply(convert_raw_gyr_to_rads).astype("float32")
            df['gyro_y'] = df['gyro_y'].apply(convert_raw_gyr_to_rads).astype("float32")
            df['gyro_z'] = df['gyro_z'].apply(convert_raw_gyr_to_rads).astype("float32")

            # Downsample each timestamp group to a maximum of 1500 rows
            def downsample_group(group):
                n = len(group)
                if n >= SEQ_LENGTH:
                    indices = np.linspace(0, n - 1, num=SEQ_LENGTH, dtype=int)
                    return group.iloc[indices]
            
            df = df.groupby('timestamp', group_keys=False).apply(downsample_group)

            labeled_data_frames[label].append(df)

dataframes = {label: pd.concat(data_frames) for label, data_frames in labeled_data_frames.items()}

# Make a copy of all the dataframes, add a label column, and concatenate them into a single dataframe
labeled_data = pd.concat([df.assign(label=label) for label, df in dataframes.items()])
labeled_data.reset_index(drop=True, inplace=True)

# Print info and overall stats about the dataset
print(labeled_data.info())
print(labeled_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572331 entries, 0 to 572330
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         572331 non-null  int64         
 1   sensor     572331 non-null  int64         
 2   acc_x      572331 non-null  float32       
 3   acc_y      572331 non-null  float32       
 4   acc_z      572331 non-null  float32       
 5   gyro_x     572331 non-null  float32       
 6   gyro_y     572331 non-null  float32       
 7   gyro_z     572331 non-null  float32       
 8   timestamp  572331 non-null  datetime64[ns]
 9   label      572331 non-null  int64         
dtypes: datetime64[ns](1), float32(6), int64(3)
memory usage: 30.6 MB
None
                  id         sensor          acc_x          acc_y  \
count  572331.000000  572331.000000  572331.000000  572331.000000   
mean    33363.663066       1.501467      -0.274573       0.085277   
min      1624.000000       1.00

### Sequence Generation and Data Splitting
After preprocessing the data, we generate sequences that will be used for training the models. These sequences are created by filling a buffer with `SEG_LENGTH` contiguous readings. Once the sequences are generated, we split the data into training, validation and test sets.

In [3]:
# Calculate the minimum length of all dataframes
min_length = min(len(data) for data in dataframes.values())

# Adjust the minimum length to be a multiple of SEQ_LENGTH
min_length = min_length - (min_length % SEQ_LENGTH)

# Crop dataframes to the adjusted minimum length
for label, data in dataframes.items():
    dataframes[label] = data.iloc[:min_length]

# Crear secuencias y etiquetas
sequences = []
labels = []

# Create sequences and labels
for label, data in dataframes.items():
    for i in range(0, len(data), SEQ_LENGTH):
        seq = data.iloc[i:i + SEQ_LENGTH]
        sequences.append(seq[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']].values)
        labels.append(label)

sequences, labels = np.array(sequences), np.array(labels)
# Print the number of sequences to be fed into the model
print(f"Sequences: {sequences.shape}")
print(f"Labels: {labels.shape}")

# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(sequences, labels, test_size=0.6, random_state=random.randint(0, 100))
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=random.randint(0, 100))

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(LABELS))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(LABELS))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(LABELS))


Sequences: (16904, 25, 6)
Labels: (16904,)


### Model Architecture

In [4]:
model_version = {
        "large": {
            "conv1d": {
                "filters": [128, 256, 128],
                "kernel_size": 3,
                "padding": "same",
            },
            "lstm": {
                "units": 64,
            },
            "dense": {
                "units": [128, 64],
            }
        },
        "mid": {
            "conv1d": {
                "filters": [64, 128, 64],
                "kernel_size": 3,
                "padding": "same",
            },
            "lstm": {
                "units": 32,
            },
            "dense": {
                "units": [64, 32],
            }
        },
        "tiny": {
            "conv1d": {
                "filters": [32, 64, 32],
                "kernel_size": 3,
                "padding": "same",
            },
            "lstm": {
                "units": 16,
            },
            "dense": {
                "units": [32, 16],
            }
        }
    }

In [5]:
def create_model(input_shape, num_classes, activation='relu', version='large'):
    # Get the model parameters from the version specified
    model_params = model_version[version]

    # Input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Reshape input for parallel paths
    reshaped_input = tf.keras.layers.Reshape(target_shape=(-1, input_shape[-1]))(inputs)
    
    # Path 1: Convolutional feature extraction
    x = reshaped_input
    conv_filters = model_params['conv1d']['filters']
    kernel_size = model_params['conv1d']['kernel_size']
    padding = model_params['conv1d']['padding']
    for filters in conv_filters:
        x = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, padding=padding, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4))(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(activation)(x)
    cnn_features = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    # Path 2: LSTM-based sequence modeling
    y = tf.keras.layers.Reshape((-1, input_shape[-1]))(inputs)
    lstm_units = model_params['lstm']['units']
    y = tf.keras.layers.LSTM(lstm_units, return_sequences=True)(y)
    y = tf.keras.layers.Dropout(0.5)(y)
    lstm_features = tf.keras.layers.GlobalAveragePooling1D()(y)
    
    # Concatenate both feature paths
    concatenated_features = tf.keras.layers.Concatenate()([cnn_features, lstm_features])
    
    # Fully connected layers
    z = concatenated_features
    dense_units = model_params['dense']['units']
    for units in dense_units:
        z = tf.keras.layers.Dense(units, activation=activation, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-5, l2=1e-4))(z)
        z = tf.keras.layers.Dropout(0.5)(z)
    
    # Output layer
    output = tf.keras.layers.Dense(num_classes, activation='softmax')(z)
    
    # Create model
    model = tf.keras.models.Model(inputs=inputs, outputs=output)
    
    return model

### ESN-PdM Case Study Models
The models employed in the case study for the ESN-PdM framework are implemented in this section.
- **Cloud Model:** Correponds to the `large` version of the model. Since the cloud has more computational resources, we can afford to use a larger model and do not require further optimization.
- **Gateway Model:** Corresponds to the `mid` version of the model. After training, we fine-tune the model using prunning and convert to LiteRT format using dynamic-range quantization.
- **Sensor Model:** Corresponds to the `small` version of the model. After training, we fine-tune the model using prunning and convert to LiteRT format using full integer quantization.

#### Cloud Model

In [6]:
"""
Model Training
"""

input_shape = (SEQ_LENGTH, 6)
num_classes = len(LABELS)
version = 'large'
batch_size = 32
epochs = 30
epochs_without_improvement = 3
initial_learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,  # Initial learning rate
    beta_1=0.9,                           # Default value
    beta_2=0.999,                         # Default value
    epsilon=1e-07,                        # Default value
    amsgrad=False                         # Default value
)
activation = 'relu'

# Train the model
cloud_model = create_model(input_shape=input_shape, num_classes=num_classes, activation=activation, version=version)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=epochs_without_improvement, restore_best_weights=True)

with tf.device('/GPU:0'):
    cloud_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = cloud_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stopping]
    )

# Evaluate the cloud_model
cloud_model_test_loss, cloud_model_test_accuracy = cloud_model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test cloud model test loss: {cloud_model_test_loss:.4f}")
print(f"Test cloud model test accuracy: {cloud_model_test_accuracy:.4f}")

# Save the cloud_model
cloud_model.save(f'{BUILD_FOLDER}/cloud_model.keras')

2024-09-25 17:28:40.654554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-25 17:28:40.657275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-25 17:28:40.657298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-25 17:28:40.660411: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-09-25 17:28:40.660437: I tensorflow/compile

127/127 [==============================] - 1s 6ms/step - loss: 0.1048 - accuracy: 0.9926
Test cloud model test loss: 0.1048
Test cloud model test accuracy: 0.9926


#### Gateway Model

In [7]:
"""
Model Training
"""

input_shape = (SEQ_LENGTH, 6)
num_classes = len(LABELS)
version = 'mid'
batch_size = 32
epochs = 30
epochs_without_improvement = 3
initial_learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,  # Initial learning rate
    beta_1=0.9,                           # Default value
    beta_2=0.999,                         # Default value
    epsilon=1e-07,                        # Default value
    amsgrad=False                         # Default value
)
activation = 'relu'

# Train the model
gateway_model = create_model(input_shape=input_shape, num_classes=num_classes, activation=activation, version=version)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=epochs_without_improvement, restore_best_weights=True)

with tf.device('/GPU:0'):
    gateway_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = gateway_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stopping]
    )

# Evaluate the gateway_model
gateway_model_test_loss, gateway_model_test_accuracy = gateway_model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test gateway model test loss: {gateway_model_test_loss:.4f}")
print(f"Test gateway model test accuracy: {gateway_model_test_accuracy:.4f}")

127/127 [==============================] - 1s 6ms/step - loss: 0.1170 - accuracy: 0.9874
Test gateway model test loss: 0.1170
Test gateway model test accuracy: 0.9874


In [8]:
"""
Fine-tuning with Pruning
"""

# Prune the gateway_model
fine_tune_epochs = 2

num_sequences = X_train.shape[0]
end_step = np.ceil(num_sequences / batch_size).astype(np.int32) * fine_tune_epochs
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.25,
        final_sparsity=0.50,
        begin_step=0,
        end_step=end_step
    )
}

pruned_gateway_model = tfmot.sparsity.keras.prune_low_magnitude(gateway_model, **pruning_params)
logdir = tempfile.mkdtemp()
with tf.device('/GPU:0'):
    pruned_gateway_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    pruned_gateway_model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=fine_tune_epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[
                tfmot.sparsity.keras.UpdatePruningStep(),
                tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
            ]
        )

# Evaluate the pruned gateway_model
pruned_gateway_model_test_loss, pruned_gateway_model_test_accuracy = pruned_gateway_model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test pruned gateway model test loss: {pruned_gateway_model_test_loss:.4f}")
print(f"Test pruned gateway model test accuracy: {pruned_gateway_model_test_accuracy:.4f}")

# Remove pruning wrappers from the pruned model
pruned_gateway_model = tfmot.sparsity.keras.strip_pruning(pruned_gateway_model)

# Save the pruned gateway_model as a concrete function
run_model = tf.function(lambda x: gateway_model(x))
_batch_size, _steps, _input_size = 1, SEQ_LENGTH, 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([_batch_size, _steps, _input_size], gateway_model.inputs[0].dtype))

# gateway_model directory.
GATEWAY_MODEL_DIR = "pruned_gateway_model_concrete_func"
gateway_model.save(GATEWAY_MODEL_DIR, save_format="tf", signatures=concrete_func)


2024-09-25 17:30:52.035136: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-25 17:30:52.036033: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-25 17:30:52.036511: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-25 17:30:52.036732: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-25 17:30:52.037228: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-09-25

127/127 [==============================] - 1s 6ms/step - loss: 0.1194 - accuracy: 0.9805
Test pruned gateway model test loss: 0.1194
Test pruned gateway model test accuracy: 0.9805
INFO:tensorflow:Assets written to: pruned_gateway_model_concrete_func/assets


INFO:tensorflow:Assets written to: pruned_gateway_model_concrete_func/assets


In [9]:
"""
Model Conversion to LiteRT
"""

converter = tf.lite.TFLiteConverter.from_saved_model(GATEWAY_MODEL_DIR)

# Dynamic range quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Save the converted model to file
tflite_model_file = 'gateway_model.tflite'
with open(f'{BUILD_FOLDER}/{tflite_model_file}', 'wb') as f:
    f.write(tflite_model)

2024-09-25 17:31:10.862363: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-25 17:31:10.862404: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-25 17:31:10.862628: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: pruned_gateway_model_concrete_func
2024-09-25 17:31:10.867230: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-25 17:31:10.867249: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: pruned_gateway_model_concrete_func
2024-09-25 17:31:10.877805: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2024-09-25 17:31:10.881688: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-25 17:31:10.952008: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bund

#### Sensor Model

In [10]:
"""
Model Training
"""

input_shape = (SEQ_LENGTH, 6)
num_classes = len(LABELS)
version = 'tiny'
batch_size = 64
epochs = 30
epochs_without_improvement = 3
initial_learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(
    learning_rate=initial_learning_rate,  # Initial learning rate
    beta_1=0.9,                           # Default value
    beta_2=0.999,                         # Default value
    epsilon=1e-07,                        # Default value
    amsgrad=False                         # Default value
)
activation = 'relu'

# Train the model
sensor_model = create_model(input_shape=input_shape, num_classes=num_classes, activation=activation, version=version)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=epochs_without_improvement, restore_best_weights=True)

with tf.device('/GPU:0'):
    sensor_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = sensor_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0,
        callbacks=[early_stopping]
    )

# Evaluate the sensor_model
sensor_model_test_loss, sensor_model_test_accuracy = sensor_model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test sensor model test loss: {sensor_model_test_loss:.4f}")
print(f"Test sensor model test accuracy: {sensor_model_test_accuracy:.4f}")

64/64 [==============================] - 0s 6ms/step - loss: 0.0665 - accuracy: 0.9914
Test sensor model test loss: 0.0665
Test sensor model test accuracy: 0.9914


In [11]:
"""
Fine-tuning with Pruning
"""

# Prune the sensor_model
fine_tune_epochs = 2

num_sequences = X_train.shape[0]
end_step = np.ceil(num_sequences / batch_size).astype(np.int32) * fine_tune_epochs
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.25,
        final_sparsity=0.50,
        begin_step=0,
        end_step=end_step
    )
}

pruned_sensor_model = tfmot.sparsity.keras.prune_low_magnitude(sensor_model, **pruning_params)
logdir = tempfile.mkdtemp()
with tf.device('/GPU:0'):
    pruned_sensor_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    pruned_sensor_model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=fine_tune_epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[
                tfmot.sparsity.keras.UpdatePruningStep(),
                tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
            ]
        )

# Evaluate the pruned sensor_model
pruned_sensor_model_test_loss, pruned_sensor_model_test_accuracy = pruned_sensor_model.evaluate(X_test, y_test, batch_size=batch_size)
print(f"Test pruned sensor model test loss: {pruned_sensor_model_test_loss:.4f}")
print(f"Test pruned sensor model test accuracy: {pruned_sensor_model_test_accuracy:.4f}")

# Remove pruning wrappers from the pruned model
pruned_sensor_model = tfmot.sparsity.keras.strip_pruning(pruned_sensor_model)

# Save the pruned sensor_model as a concrete function
run_model = tf.function(lambda x: sensor_model(x))
_batch_size, _steps, _input_size = 1, SEQ_LENGTH, 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([_batch_size, _steps, _input_size], sensor_model.inputs[0].dtype))

# Sensor_model directory.
SENSOR_MODEL_DIR = "pruned_sensor_model_concrete_func"
sensor_model.save(SENSOR_MODEL_DIR, save_format="tf", signatures=concrete_func)

64/64 [==============================] - 0s 5ms/step - loss: 0.1554 - accuracy: 0.9677
Test pruned sensor model test loss: 0.1554
Test pruned sensor model test accuracy: 0.9677
INFO:tensorflow:Assets written to: pruned_sensor_model_concrete_func/assets


INFO:tensorflow:Assets written to: pruned_sensor_model_concrete_func/assets


In [12]:
"""
Model Conversion to LiteRT
"""
converter = tf.lite.TFLiteConverter.from_saved_model(SENSOR_MODEL_DIR)

# Representative dataset for quantization
def representative_dataset():
    for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
        yield [input_value]

# Full integer quantization 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS_INT8
]
converter.representative_dataset = representative_dataset
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model = converter.convert()

# Save the converted model to file
tflite_model_file = 'sensor_model.tflite'
with open(f'{BUILD_FOLDER}/{tflite_model_file}', 'wb') as f:
    f.write(tflite_model)

2024-09-25 17:31:58.263038: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-09-25 17:31:58.263089: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-09-25 17:31:58.263217: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: pruned_sensor_model_concrete_func
2024-09-25 17:31:58.269251: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-09-25 17:31:58.269275: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: pruned_sensor_model_concrete_func
2024-09-25 17:31:58.286763: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-09-25 17:31:58.356187: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: pruned_sensor_model_concrete_func
2024-09-25 17:31:58.386141: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load fo

### Model Comparison
Below me compare the performance of the cloud, gateway, and sensor models in terms of accuracy and size. The accuracy is evaluated using the test set, while the size is measured in terms of the compressed model file size.

In [13]:
# Size comparison
def get_gzipped_model_size(file):
  import os, zipfile, tempfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)

def format_size(size_bytes):
    if size_bytes < 1024:
        return f"{size_bytes:.2f} bytes"
    elif size_bytes < 1024 ** 2:
        return f"{size_bytes / 1024:.2f} KB"
    else:
        return f"{size_bytes / 1024 ** 2:.2f} MB"

In [14]:

# Accuracy comparison
def _quantize_model(input_data, input_details):
    input_scale, input_zero_point = input_details["quantization"]
    input_data = input_data / input_scale + input_zero_point
    return input_data


def evaluate_tflite_model(tflite_model_path, X_test, y_test):
    # Load the TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    correct_predictions = 0
    
    for i in range(X_test.shape[0]):
        # Check if the input type is quantized, then rescale input data to uint8
        input_data = _quantize_model(X_test[i], input_details) if input_details['dtype'] == np.uint8 else X_test[i]

        input_data = np.expand_dims(input_data, axis=0).astype(input_details['dtype'])   
        interpreter.set_tensor(input_details['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details['index'])
        
        predicted_label = np.argmax(output_data)


        true_label = np.argmax(y_test[i])

        if predicted_label == true_label:
            correct_predictions += 1

    return correct_predictions / X_test.shape[0]

def evaluate_keras_model(keras_model_path, X_test, y_test, batch_size=32):
    model = tf.keras.models.load_model(keras_model_path)
    _, test_accuracy = model.evaluate(X_test, y_test, batch_size)
    return test_accuracy


from sklearn.metrics import recall_score

def compute_recall_per_class_tflite(tflite_model_path, X_test, y_test):
    # Load the TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output tensor details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    y_true = []
    y_pred = []
    
    for i in range(X_test.shape[0]):
        # Check if the input type is quantized, then rescale input data to uint8
        input_data = _quantize_model(X_test[i], input_details) if input_details['dtype'] == np.uint8 else X_test[i]

        input_data = np.expand_dims(input_data, axis=0).astype(input_details['dtype'])   
        interpreter.set_tensor(input_details['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details['index'])
        
        predicted_label = np.argmax(output_data)
        true_label = np.argmax(y_test[i])

        y_true.append(true_label)
        y_pred.append(predicted_label)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Compute recall for each class
    recalls = recall_score(y_true, y_pred, average=None)
    cm = tf.math.confusion_matrix(y_true, y_pred)
    recall_per_class = {f'Class {i}': recalls[i] for i in range(len(recalls))}
    return cm, recall_per_class

def compute_recall_per_class_keras(keras_model_path, X_test, y_test, batch_size=32):
    model = tf.keras.models.load_model(keras_model_path)
    y_pred = model.predict(X_test, batch_size=batch_size)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Compute recall for each class
    recalls = recall_score(y_true, y_pred, average=None)
    cm = tf.math.confusion_matrix(y_true, y_pred)
    recall_per_class = {f'Class {i}': recalls[i] for i in range(len(recalls))}
    return cm, recall_per_class

def normalize_cm(cm):
    return cm / cm.sum(axis=1)[:, np.newaxis]



In [16]:
# Split data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(sequences, labels, test_size=0.6, random_state=random.randint(0, 100))
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=random.randint(0, 100))

# Convert labels to categorical format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(LABELS))
y_val = tf.keras.utils.to_categorical(y_val, num_classes=len(LABELS))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(LABELS))

path_to_cloud_model = os.path.join(BUILD_FOLDER, "cloud_model.keras")
path_to_gateway_model = os.path.join(BUILD_FOLDER, "gateway_model.tflite")
path_to_sensor_model = os.path.join(BUILD_FOLDER, "sensor_model.tflite")

cloud_model_size = os.path.getsize(path_to_cloud_model)
gateway_model_size = os.path.getsize(path_to_gateway_model)
sensor_model_size = os.path.getsize(path_to_sensor_model)

cloud_model_gzipped_size = get_gzipped_model_size(path_to_cloud_model)
gateway_model_gzipped_size = get_gzipped_model_size(path_to_gateway_model)
sensor_model_gzipped_size = get_gzipped_model_size(path_to_sensor_model)

cloud_model_accuracy = evaluate_keras_model(path_to_cloud_model, X_test, y_test)
gateway_model_accuracy = evaluate_tflite_model(path_to_gateway_model, X_test, y_test)
sensor_model_accuracy = evaluate_tflite_model(path_to_sensor_model, X_test, y_test)

cloud_cm, cloud_recall = compute_recall_per_class_keras(path_to_cloud_model, X_test, y_test)
gateway_cm, gateway_recall = compute_recall_per_class_tflite(path_to_gateway_model, X_test, y_test)
sensor_cm, sensor_recall = compute_recall_per_class_tflite(path_to_sensor_model, X_test, y_test)

cloud_norm_cm = normalize_cm(np.array(cloud_cm))
gateway_norm_cm = normalize_cm(np.array(gateway_cm))
sensor_norm_cm = normalize_cm(np.array(sensor_cm))


print("\n---------------------- Model Sizes ----------------------")
print("Size of cloud Keras model: %s" % format_size(cloud_model_size))
print("Size of gateway Tflite model: %s" % format_size(gateway_model_size))
print("Size of sensor Tflite model: %s" % format_size(sensor_model_size))

print("\n---------------------- Gzipped Model Sizes ----------------------")
print("Size of gzipped cloud Keras model: %s" % format_size(cloud_model_gzipped_size))
print("Size of gzipped gateway Tflite model: %s" % format_size(gateway_model_gzipped_size))
print("Size of gzipped sensor Tflite model: %s" % format_size(sensor_model_gzipped_size))

print("\n---------------------- Model Size Comparison ----------------------")
print("Gateway model is %.2f times smaller than the cloud model" % (cloud_model_gzipped_size / gateway_model_gzipped_size))
print("Sensor model is %.2f times smaller than the cloud model" % (cloud_model_gzipped_size / sensor_model_gzipped_size))
print("Sensor model is %.2f times smaller than the gateway model" % (gateway_model_gzipped_size / sensor_model_gzipped_size))

print("\n---------------------- Model Accuracies ----------------------")
print(f"Cloud model accuracy: {cloud_model_accuracy * 100:.2f}% [Tensorflow Keras]")
print(f"Gateway model accuracy: {gateway_model_accuracy * 100:.2f}% [TensorFlow Lite]")
print(f"Sensor model accuracy: {sensor_model_accuracy * 100:.2f}% [TensorFlow Lite Micro]")

print("\n---------------------- Model Recall ----------------------")
print("Cloud model recall per class:" , cloud_recall)
print("Gateway model recall per class:" , gateway_recall)
print("Sensor model recall per class:" , sensor_recall)

127/127 [==============================] - 1s 4ms/step

---------------------- Model Sizes ----------------------
Size of cloud Keras model: 2.97 MB
Size of gateway Tflite model: 90.00 KB
Size of sensor Tflite model: 30.07 KB

---------------------- Gzipped Model Sizes ----------------------
Size of gzipped cloud Keras model: 2.69 MB
Size of gzipped gateway Tflite model: 55.51 KB
Size of gzipped sensor Tflite model: 15.87 KB

---------------------- Model Size Comparison ----------------------
Gateway model is 49.57 times smaller than the cloud model
Sensor model is 173.38 times smaller than the cloud model
Sensor model is 3.50 times smaller than the gateway model

---------------------- Model Accuracies ----------------------
Cloud model accuracy: 99.38% [Tensorflow Keras]
Gateway model accuracy: 94.09% [TensorFlow Lite]
Sensor model accuracy: 91.40% [TensorFlow Lite Micro]

---------------------- Model Recall ----------------------
Cloud model recall per class: {'Class 0': 0.981132075